# REFERENCE
https://github.com/hypnopump/pysimplechain/blob/master/simple_chain.py

In [1]:
counter=0
utxo_pool={}
Transaction_queue=[]
users={}
user_counter=0

# error classes

In [2]:
class InvalidTransaction(Exception):
	def __init__(self,*args,**kwargs):
		Exception.__init__(self,*args,**kwargs)

class InvalidBlock(Exception):
	def __init__(self,*args,**kwargs):
		Exception.__init__(self,*args,**kwargs)

class InvalidBlockchain(Exception):
	def __init__(self,*args,**kwargs):
		Exception.__init__(self,*args,**kwargs)
class InvalidTransactionAttempt(Exception):
	def __init__(self,*args,**kwargs):
		Exception.__init__(self,*args,**kwargs)

# transaction inputs (tin and tout)

In [3]:
import datetime
import hashlib
import time

# single transaction (utxo)
class transaction:
  def __init__(self, sender_id,receiver_id,amt,id):
    self.sender_id=sender_id
    self.receiver_id=receiver_id
    self.amt=amt
    # self.age=0
    # self.p_id=
    self.id=id
  def __repr__(self):
    return 'Transaction<sender: {}, receiver: {}, amt: {}>'.format(
      self.sender_id, self.receiver_id, self.amt
    )
  def setamt(self,amt):
    retval=self.amt-amt
    self.amt=amt
    return retval



# Transactions

In [4]:
# collection of utxos Transaction
class Transaction:
  def __init__(self, utxos,gen_utxos):
    self.utxos=utxos
    self.new_utxos=gen_utxos
    self.hash = None
    self.prev_hash = "0"*64
    self.timestamp = time.time()
    self.data = ""
    for i in self.utxos:
      self.data=hashlib.sha256(bytearray(str(i.id)+str(self.data), "utf-8")).hexdigest()

    self.payload_hash = self._hash_payload()

  def _hash_payload(self):
    return hashlib.sha256(bytearray(str(self.timestamp) + str(self.data), "utf-8")).hexdigest()

  def _hash_Transaction(self):
    return hashlib.sha256(bytearray(str(self.prev_hash) + self.payload_hash, "utf-8")).hexdigest()

  def link(self, Tr):
    """ Link the transaction to the previous one via hashes."""
    self.prev_hash = Tr.hash

  def seal(self):
    """ Get the transaction hash. """
    self.hash = self._hash_Transaction()


  def validate(self):
    """ Check whether the transaction is valid or not. """
    if self.payload_hash != self._hash_payload():
      raise InvalidTransaction("Invalid payload hash in message: " + str(self))
    if self.hash != self._hash_Transaction():
      raise InvalidTransaction("Invalid message hash in message: " + str(self))

  def __repr__(self):
    return 'Transaction<hash: {}, prev_hash: {}, utxo_hash: {}>'.format(
      self.hash, self.prev_hash, self.data[:20]
    )

# Block

In [5]:
class Block:
  def __init__(self, *args):
    self.Trxs = []
    self.timestamp = None
    self.prev_hash = None
    self.hash = None
    self.nonce=0
    # if args:
    #   for arg in args:
    #     self.add_message(arg)

  def _hash_block(self):
    return hashlib.sha256(bytearray(str(self.prev_hash) + str(self.timestamp) + self.Trxs[-1].hash + str(self.nonce), "utf-8")).hexdigest()

  def add_transaction(self, Trx):
    if len(self.Trxs) > 0:
      Trx.link(self.Trxs[-1])
    Trx.seal()
    Trx.validate()
    self.Trxs.append(Trx)

  def link(self, block):
    self.prev_hash = block.hash

  def seal(self):
    diff=2
    self.timestamp = time.time()
    self.hash = self._hash_block()
    while(self.hash[:diff]!="0"*diff):
      self.nonce+=1
      self.hash = self._hash_block()


  def validate(self):
    for i, msg in enumerate(self.Trxs):
      try:
        msg.validate()
        if i > 0 and msg.prev_hash != self.Trxs[i-1].hash:
          raise InvalidBlock("Invalid block: Message #{} has invalid message link in block: {}".format(i, str(self)))
      except InvalidBlock as ex:
        raise InvalidBlock("Invalid block: Message #{} failed validation: {}. In block: {}".format(
        i, str(ex), str(self))
        )

  def __repr__(self):
    return 'Block<hash: {}, prev_hash: {}, Trxs: {}, time: {}>'.format(
    self.hash, self.prev_hash, len(self.Trxs), self.timestamp
  )

# BlockChain

In [6]:
class BlockChain:
	def __init__(self):
		self.chain = []

	def add_block(self, block):
		""" Add a block if valid."""
		if len(self.chain) > 0:
			block.prev_hash = self.chain[-1].hash
		block.seal()
		block.validate()
		self.chain.append(block)

	def validate(self):
		""" Validates each block, in order.
			An invalid block invalidates the chain.
		"""
		for i, block in enumerate(self.chain):
			try:
				block.validate()
			except InvalidBlock as exc:
				raise InvalidBlockchain("Invalid blockchain at block number {} caused by: {}".format(i, str(exc)))
		return True

	def __repr__(self):
		return 'BlockChain<blocks: {}>'.format(len(self.chain))




# User

In [7]:
class User:
  def __init__(self,name,id):
    self.name=name
    self.id=id

  def transact(self,receiver_ids,amnts):
    global counter
    global utxo_pool
    global Transaction_queue
    global users
    global user_counter
    val=0
    t_in=[]
    amnt=sum(amnts)
    for i in utxo_pool:
      if utxo_pool[i].receiver_id==self.id:
        val+=utxo_pool[i].amt
        t_in.append(utxo_pool[i])
        if val>=amnt:
          break
    if val<amnt:
      raise InvalidTransactionAttempt("not enough utxos in pool: " + str(self))

    t_out=[]
    for i,j in zip(receiver_ids,amnts):
      t_out.append(transaction(self.id,i,j,counter))
      counter+=1

    if val>amnt:
      t_out.append(transaction(self.id,self.id,val-amnt,counter))
      counter+=1
    Transaction_queue.append(Transaction(t_in,t_out))
  
  def mine(self,chain):
    global counter
    global utxo_pool
    global Transaction_queue
    global users
    global user_counter
    
    block=Block()
    mining_tr=[]
    mining_tr.append(transaction("_BANK",self.id,6.25,counter))
    counter+=1
    block.add_transaction(Transaction([],mining_tr))
    i=0
    while i<10 and i<len(Transaction_queue):
      temp_Tr=Transaction_queue[i]
      # print(i)
      processing_trans=[]
      for trx in temp_Tr.new_utxos:
        processing_fee=trx.setamt(trx.amt*0.95)
        processing_trans.append(transaction(trx.sender_id,self.id,processing_fee,counter))
        counter+=1
      for ptr in processing_trans:
        temp_Tr.new_utxos.append(ptr)
      
      # add this in the block
      block.add_transaction(temp_Tr)
      i+=1
    
    chain.add_block(block)
      # validate the block
    
    j=0
    # pick first 10 Transactions in the queue
    while j<10 and j<len(Transaction_queue):
      temp_Tr=Transaction_queue[j]   
      #  remove the old utxos
      rem_utxo_id=[]
      for utxo1 in temp_Tr.utxos:
        for i in utxo_pool:
          if i == utxo1.id:
            rem_utxo_id.append(i)
            break
      # utxo_pool = [x for x in utxo_pool if not x.id in rem_utxo_id]
      for i in rem_utxo_id:
        del utxo_pool[i]
      # add the new ones
      for i in temp_Tr.new_utxos:
        utxo_pool[i.id]=i
      j+=1
    utxo_pool[mining_tr[0].id]=mining_tr[0]
    # remove the Transactions from the Transactionsqueue
    i=0
    while i<10 and Transaction_queue:
      Transaction_queue.pop(0)
      i+=1

# Run

In [8]:
def manager():
  global counter
  global utxo_pool
  global Transaction_queue
  global users
  global user_counter
  counter=0
  utxo_pool={}
  Transaction_queue=[]
  users={}
  user_counter=0

  chain = BlockChain()

  msg = """
  Basic implementation of a Blockchain. Changes are inmutable. Be aware.
  Action set:
  - add User                           (0)
  - show a users utxos in the pool     (1)
  - transact                           (2)
  - select user to mine                (3)
  - show a block (index will be asked) (4)
  - show the whole chain               (5)
  - validate the chain integrity       (6)
  - display the global pool            (7)
  - display the transaction queue      (8)
  - display users list                 (9)
  - exit the program                   (10)

  The validate action will kill the program if the integrity if the chain
  is compromised.
  """

  print(msg)	
  while True:
    print()

    decide = input("Your action: ")

    if decide == "0":
      name=input("Provide name")
      users[user_counter]=User(name,user_counter)
      user_counter= user_counter+1
      print("User create with user id ", user_counter-1)

    elif decide == "1":
      id=int(input("Provide id"))
      for i in utxo_pool:
        if utxo_pool[i].receiver_id == id :
          print(utxo_pool[i])


    elif decide == "2":
      user_id=int(input("Provide user id"))

      print("provide txn_outs,enter -1 to break")
      recids=[]
      amts=[]
      while True:
        recid=int(input("recv id"))
        if recid == -1:
          break
        recids.append(recid)
        amts.append(float(input("amt")))
        
      users[user_id].transact(recids,amts)
      print("Transaction added by user" , user_id)

    elif decide == "3":
      id=int(input("Provide id"))
      users[id].mine(chain)
      print("Block mined by user ",id) 

    elif decide == "4":
      index = int(input("Provide the index: "))
      if len(chain.chain)>0:
        try: print(chain.chain[index])
        except: print("An issue occurred")
    elif decide == "5":
      for b in chain.chain:
        print(b)
        print("----------------")
    elif decide == "6":
      if chain.validate(): print("Integrity validated.")
    elif decide == "7":
      for utxo in utxo_pool:
        print(utxo,utxo_pool[utxo])
      print("----------------")
    elif decide == "8":
      for trx in Transaction_queue:
        print(trx)
      print("----------------")
    elif decide == "9":
      for userid in users:
        print(userid,users[userid].name)
      print("----------------")
    else:
      break

if __name__ == "__main__":
	manager()


  Basic implementation of a Blockchain. Changes are inmutable. Be aware.
  Action set:
  - add User                           (0)
  - show a users utxos in the pool     (1)
  - transact                           (2)
  - select user to mine                (3)
  - show a block (index will be asked) (4)
  - show the whole chain               (5)
  - validate the chain integrity       (6)
  - display the global pool            (7)
  - display the transaction queue      (8)
  - display users list                 (9)
  - exit the program                   (10)

  The validate action will kill the program if the integrity if the chain
  is compromised.
  

Your action: 0
Provide nameA
User create with user id  0

Your action: 0
Provide nameB
User create with user id  1

Your action: 0
Provide nameC
User create with user id  2

Your action: 3
Provide id0
Block mined by user  0

Your action: 2
Provide user id0
provide txn_outs,enter -1 to break
recv id1
amt3
recv id-1
Transaction added by user